## Семинар 1 Индекс

## Intro

### работа с файлами и папками

In [1]:
import os

curr_dir = os.getcwd()
filepath = os.path.join(curr_dir, 'test.txt')

### os.path  
путь до файла

In [25]:
# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath(filepath))


# возвращает имя файла / папки по полному ти до него
print(os.path.basename(filepath))


# проверить существование директории - True / False
print(os.path.exists(curr_dir))

### os.listdir  
возвращает список файлов в данной директории

In [24]:
os.listdir(curr_dir)

При обходе файлов не забывайте исключать системные директории, такие как .DS_Store

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [21]:
for root, dirs, files in os.walk(curr_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [23]:
list(os.walk(curr_dir))

### чтение файла 

In [1]:
filepath = 'test.txt'


# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

    
#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

    
#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 

Напоминание про enumerate:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, не надо так

##  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзьей. Одна серия - один документ.

**To do:** Постройте небольшой модуль поискового движка, который сможет осуществлять поиск по коллекции документов.
На входе запрос и проиндексированная коллекция (в том виде, как посчитаете нужным), на выходе отсортированный по релевантности с запросом список документов коллекции. 

Релизуйте:
    - функцию препроцессинга данных
    - функцию индексирования данных
    - функцию метрики релевантности 
    - собственно, функцию поиска

[download_friends_corpus](https://yadi.sk/d/yVO1QV98CDibpw)

Напоминание про defaultdict: 
> В качестве multiple values словаря рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [1]:
### _check : в коллекции должно быть около 165 файлов

С помощью обратного индекса посчитайте:  


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) какой сезон был самым популярным у Чендлера? у Моники?

e) кто из главных героев статистически самый популярный? 


In [115]:
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from math import log
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
import scipy

In [22]:
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

In [28]:
vectorizer = CountVectorizer()

In [23]:
def process(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " "\
              and token.strip() not in punctuation] 
    text = " ".join(tokens)
    return text

In [56]:
corpus = []
fil = []
for root, dirs, files in os.walk('friends'):
    for file in files:
        if file[0] != '.':
            fil.append(os.path.join(root, file))
            file = open(os.path.join(root, file), 'r', encoding='utf-8')
            text = file.read()
            text = process(text)
            corpus.append(text)         

In [106]:
def indeciese(corpus):
    X = vectorizer.fit_transform(corpus)
    Y = vectorizer.get_feature_names()
    arr = X.toarray()
    df = pd.DataFrame(data=arr, index=fil, columns=Y)
    return df

In [112]:
def relevance(data):
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    i = tfidf_transformer.fit(X)
    m = tfidf_transformer.fit_transform(X).toarray()
    return (i, m)

array([[0.03472717, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03369391, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05011025, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.0352362 , 0.        , 0.        , ..., 0.        , 0.08312966,
        0.        ],
       [0.03397161, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03495448, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [116]:
def search(quiry):
    data = indeciese(corpus)
    tfidf_transformer, tfidf_collection = relevance (data)
    quiry = process(quiry)
    quiry_ind = vectorizer.fit_transform(quiry)
    quiry_ind = tfidf_transformer.fit_transform(quiry_ind).toarray()
    tosort = []
    for i in range(len(fil)):
        q = [fil[i], scipy.spatial.distance.cosine[quiry_ind, tfidf_collection[0][i]]]
        tosort.append(q)
    return tosort.sort(key = sortSecond)  

In [121]:
maximum = 0
word = 0
alltexts = []
for col in df:
    if df[col].sum() > maximum:
        maximum = df[col].sum()
        word = col
    if df[col].sum() == 165:
        alltexts.append(col)
print ('Самое частотное слово это: ' + word)
print ('Слова, которые встречаются во всех текстах: ' + ', '.join(alltexts))

Самое частотное слово это: это
Слова, которые встречаются во всех текстах: каждый, какой


In [119]:
minimum = 500
word = 0
for col in df:
    if df[col].sum() < minimum:
        minimum = df[col].sum()
        word = col
print ('Слово, которое встречается реже всего: ' + word)

Слово, которое встречается реже всего: 03815


In [76]:
names = ['Чендлер', 'Моника']
nnames = []
for i in names:
    nnames.append(process(i))
print (nnames)

['чендлер', 'моника']


In [98]:
s2 = df['моника'][0:25].sum()
s5 = df['моника'][25:49].sum()
s4 = df['моника'][49:75].sum()
s3 = df['моника'][75:96].sum()
s6 = df['моника'][96:121].sum()
s1 = df['моника'][121:141].sum()
s7 = df['моника'][141:].sum()

s2 = df['чендлер'][0:25].sum()
s5 = df['чендлер'][25:49].sum()
s4 = df['чендлер'][49:75].sum()
s3 = df['чендлер'][75:96].sum()
s6 = df['чендлер'][96:121].sum()
s1 = df['чендлер'][121:141].sum()
s7 = df['чендлер'][141:].sum()

#Ответ: Моника - 7
#Ответ: Чендлер - 6

In [100]:
main = 'Рейчел, Моника, Фиби, Джоуи, Чендлер, Росс'
main = main.split(', ')

['Рейчел', 'Моника', 'Фиби', 'Джоуи', 'Чендлер', 'Росс']

In [117]:
ma = 0
name = 0
for nam in main:
    if df[nam].sum() > ma:
        ma = df[nam].sum()
        name = nam
print (name + ' самый популярый персонаж')

росс самый популярый персонаж
